In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
from neo4j import GraphDatabase
%run ../py/cypher_utlis.py
%run ../load_magic/storage.py
%run ../load_magic/dataframes.py

s = Storage()
cu = CypherUtilities()

Wall time: 1.52 s


In [3]:

uri = 'bolt://localhost:7687'
user = 'neo4j'
password = 'Genesis11'
driver = GraphDatabase.driver(uri, auth=(user, password))

In [54]:

INFIX_STR = ''',
    '''
def get_tx_str(column_descriptions_df, idx_column, f_str):
    mask_series = (column_descriptions_df.dtype == 'int64')
    int64s_list = column_descriptions_df[mask_series].column_name.tolist()
    with_str = INFIX_STR.join(['toInteger(row.{}) AS {}'.format(x, x) for x in int64s_list])
    objects_list = column_descriptions_df[~mask_series].column_name.tolist()
    if objects_list:
        with_str += INFIX_STR + INFIX_STR.join(['row.{} AS {}'.format(x, x) for x in objects_list])
    mask_series = (column_descriptions_df.column_name != idx_column)
    nonidxs_list = column_descriptions_df[mask_series].column_name.tolist()
    set_str = INFIX_STR.join(['x.{} = {}'.format(x, x) for x in nonidxs_list])
    tx_str = f_str.format(with_str, set_str)
    
    return tx_str


## Get Primary Keys

In [69]:

sql_str = '''
SELECT
    tab.[name] AS table_name,
    Substring(column_names, 1, Len(column_names) - 1) AS [columns]
FROM
    sys.tables tab LEFT OUTER JOIN
    sys.indexes pk ON
        tab.object_id = pk.object_id AND
        pk.is_primary_key = 1
    CROSS apply (
        SELECT col.[name] + ', '
        FROM
            sys.index_columns ic INNER JOIN
            sys.columns col ON
                ic.object_id = col.object_id AND
                ic.column_id = col.column_id
        WHERE
            ic.object_id = tab.object_id AND
            ic.index_id = pk.index_id
        ORDER  BY col.column_id
        FOR xml path ('')
        ) D (column_names)
WHERE
    Schema_name(tab.schema_id) = 'dbo' AND
    tab.[name] != 'sysdiagrams'
ORDER BY
    Schema_name(tab.schema_id),
    tab.[name];'''
pks_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))

In [99]:

import os
from shutil import copyfile

def clear_database(tx):
    delete_str = "CALL apoc.periodic.iterate('MATCH (n) RETURN n', 'DETACH DELETE n', {batchSize:1000})"
    tx.run(delete_str)
def create_constraint(tx, class_name):
    constraint_str = 'CREATE CONSTRAINT Unique{} IF NOT EXISTS ON (x:{}) ASSERT x.id IS UNIQUE;'
    constraint_str = constraint_str.format(class_name, class_name)
    tx.run(constraint_str)
def f(x):
    if type(x) is str:
        x = x.strip()
    
    return x
dbmss_dir = r'C:\Users\dev\.Neo4jDesktop\relate-data\dbmss'
import_dir = os.path.join(dbmss_dir, 'dbms-c715bff3-5764-4d71-bf04-6eceb4dd13f4', 'import')
csv_dir = '../saves/csv'

In [100]:

import pandas as pd

def do_cypher_tx(tx, cypher):
    results_list = tx.run(cypher)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))
    
    return values_list

In [101]:

with driver.session() as session:
    session.write_transaction(clear_database)
for row_index, row_series in pks_df.iterrows():
    class_name = row_series.table_name
    with driver.session() as session:
        session.write_transaction(create_constraint, class_name)
    idx_column = row_series.columns
    sql_str = 'SELECT * FROM [Jobhunting].[dbo].[{}];'.format(class_name)
    table_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
    column_descriptions_df = get_column_descriptions(table_df)
    mask_series = (column_descriptions_df.dtype == 'object')
    for column_name in column_descriptions_df[mask_series].column_name.tolist():
        table_df[column_name] = table_df[column_name].map(f)
    s.save_dataframes(**{class_name: table_df})
    csv_name = f'{class_name}.csv'
    src = os.path.join(csv_dir, csv_name)
    dst = os.path.join(import_dir, csv_name)
    copyfile(src, dst)
    formatted_tx_str = "\nLOAD CSV WITH HEADERS FROM 'file:///{}' AS row\nWITH\n    {{}}".format(csv_name)
    formatted_tx_str += '\nMERGE (x:{} {{{{{}: {}}}}}) SET'.format(class_name, idx_column, idx_column)
    formatted_tx_str += '\n    {};'
    tx_str = get_tx_str(column_descriptions_df, idx_column, formatted_tx_str)
    print(tx_str)
    with driver.session() as session:
        session.write_transaction(do_cypher_tx, tx_str)

Saving to C:\Users\dev\Documents\Repositories\job-hunting\saves\csv\FileNames.csv

LOAD CSV WITH HEADERS FROM 'file:///FileNames.csv' AS row
WITH
    toInteger(row.file_name_id) AS file_name_id,
    row.file_name AS file_name,
    row.percent_fit AS percent_fit,
    row.is_opportunity_application_emailed AS is_opportunity_application_emailed,
    row.opportunity_application_email_date AS opportunity_application_email_date,
    row.is_remote_delivery AS is_remote_delivery,
    row.manager_notes AS manager_notes
MERGE (x:FileNames {file_name_id: file_name_id}) SET
    x.file_name = file_name,
    x.percent_fit = percent_fit,
    x.is_opportunity_application_emailed = is_opportunity_application_emailed,
    x.opportunity_application_email_date = opportunity_application_email_date,
    x.is_remote_delivery = is_remote_delivery,
    x.manager_notes = manager_notes;
Saving to C:\Users\dev\Documents\Repositories\job-hunting\saves\csv\HeaderTags.csv

LOAD CSV WITH HEADERS FROM 'file:///HeaderT


## Get Foreign Keys

In [103]:

columns_list = ['FKTABLE_NAME', 'FKCOLUMN_NAME']
formatted_sql_str = '''
EXEC sp_fkeys @pktable_name = '{}', @pktable_owner = 'dbo';'''
rows_list = []
for pkrow_index, pkrow_series in pks_df.iterrows():
    pktable_name = pkrow_series.table_name
    fks_df = pd.DataFrame(su.get_execution_results(CURSOR, formatted_sql_str.format(pktable_name)))
    for fkrow_index, fkrow_series in fks_df.iterrows():
        row_dict = {}
        row_dict['pktable_name'] = pktable_name
        fktable_name = fkrow_series.FKTABLE_NAME
        row_dict['fktable_name'] = fktable_name
        fkcolumn_name = fkrow_series.FKCOLUMN_NAME
        row_dict['fkcolumn_name'] = fkcolumn_name
        rows_list.append(row_dict)
fks_df = pd.DataFrame(rows_list)

In [111]:

formatted_tx_str = '''
MATCH (a:{}), (b:{}) 
   WHERE a.{} = b.{}
CREATE (a)-[r: Relation]->(b);'''
write_str = '''
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)'''
for fkrow_index, fkrow_series in fks_df.iterrows():
    pktable_name = fkrow_series.pktable_name
    fktable_name = fkrow_series.fktable_name
    fkcolumn_name = fkrow_series.fkcolumn_name
    print('\ntx_str = ', end='"""')
    print(formatted_tx_str.format(pktable_name, fktable_name, fkcolumn_name, fkcolumn_name), end='"""')
    print(write_str)


tx_str = """
MATCH (a:FileNames), (b:HeaderTagSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (a)-[r: Relation]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

tx_str = """
MATCH (a:FileNames), (b:NavigableParentSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (a)-[r: Relation]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

tx_str = """
MATCH (a:HeaderTags), (b:HeaderTagSequence) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: Relation]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

tx_str = """
MATCH (a:HeaderTags), (b:NavigableParents) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: Relation]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

tx_str = """
MATCH (a:MinimumRequirementsSection), (b:NavigableParentSequence) 
   WHERE a.mrs_id = b.mrs_id
CRE

In [112]:

tx_str = """
MATCH (a:FileNames), (b:HeaderTagSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (b)-[r: IS_CONTAINED_IN]->(a);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [113]:

tx_str = """
MATCH (a:FileNames), (b:NavigableParentSequence) 
   WHERE a.file_name_id = b.file_name_id
CREATE (b)-[r: IS_CONTAINED_IN]->(a);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [114]:

tx_str = """
MATCH (a:HeaderTags), (b:HeaderTagSequence) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: IS_PART_OF]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [115]:

tx_str = """
MATCH (a:HeaderTags), (b:NavigableParents) 
   WHERE a.header_tag_id = b.header_tag_id
CREATE (a)-[r: SUMMARIZES]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [116]:

tx_str = """
MATCH (a:MinimumRequirementsSection), (b:NavigableParentSequence) 
   WHERE a.mrs_id = b.mrs_id
CREATE (a)-[r: SUMMARIZES]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)

In [117]:

tx_str = """
MATCH (a:NavigableParents), (b:NavigableParentSequence) 
   WHERE a.navigable_parent_id = b.navigable_parent_id
CREATE (a)-[r: IS_PART_OF]->(b);"""
with driver.session() as session:
    session.write_transaction(do_cypher_tx, tx_str)


----
## Display Nodes and Relationships

In [ ]:

TX_STR = '''
CALL {
  MATCH (a)-[r: IS_CONTAINED_IN]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
UNION
  MATCH (a)-[r: IS_PART_OF]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
UNION
  MATCH (a)-[r: SUMMARIZES]->(b)
  RETURN a, b
  ORDER BY rand()
  LIMIT 15
}
RETURN a, b;'''
def display_nodes_tx(tx):
    result = tx.run(TX_STR)
    print([f'result.{fn}' for fn in dir(result) if not fn.startswith('_')])
    record_dict = result.single()
    
    return record_dict
session = driver.session()
session.read_transaction(display_nodes_tx)

In [ ]:

TX_STR = '''
CALL {
  MATCH (x:FileNames)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:HeaderTags)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:HeaderTagSequence)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:MinimumRequirementsSection)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:NavigableParents)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:NavigableParentSequence)
  RETURN x
  LIMIT 10
UNION
  MATCH (x:PartsOfSpeech)
  RETURN x
  LIMIT 10
}
RETURN x;'''
def display_nodes_tx(tx):
    result = tx.run(TX_STR)
    print([f'result.{fn}' for fn in dir(result) if not fn.startswith('_')])
    record_dict = result.single()
    
    return record_dict
session = driver.session()
session.read_transaction(display_nodes_tx)